In [1]:
# Use Selenium to render the webpage and scroll through & BeautifulSoup to capture info from the page.

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bsoup
import pandas as pd
import time
import json
import requests


In [2]:
# Parse indiividual profiles seperately
def sub_catcher(priv_url):
    if priv_url[:5] != 'https':
        priv_url = 'https://www.practo.com' + priv_url
    
    priv_browser = requests.get(priv_url)

    priv_page_soup = bsoup(priv_browser.text, "html.parser")
    priv_container = priv_page_soup.body.findAll("script")
    
    # Convert data in JSON and save
    priv_data_json = json.loads(priv_container[0].string)
    sub_data_json = priv_data_json[0]
    
    try:
        claim_check = priv_page_soup.body.findAll('span', {'data-for': 'React-tooltip-claim'})[0].text
    except:
        claim_check = ''
    
    qualification = priv_page_soup.body.findAll('p', {'class': 'c-profile__details'})[0].text
    
    try:
        verification_detail = priv_page_soup.body.findAll('span', {'data-qa-id': 'doctor-verification-label'})[0].text
    except:
        verification_detail = 'Not Verified'
    
    
    try:
        profile_score = priv_page_soup.body.findAll('span', {'class': 'u-green-text u-bold u-large-font'})[0].text
    except:
        profile_score = ''
    
    try:
        profile_votes = priv_page_soup.body.findAll('span', {'class': 'u-smallest-font u-grey_3-text'})[0].text
    except:
        profile_votes = ''

    exp_years = priv_page_soup.body.findAll('div', {'class': 'c-profile__details'})
    exp_years = exp_years[0].findAll('h2')[-1].text
    
    for elmnt in ['education', 'memberships', 'experience', 'registrations', 'trainings and certifications']:
        sub_d = priv_page_soup.body.findAll('div', {'id': elmnt})
        
        if sub_d != []:
            sub_d = sub_d[0].findAll('div', {'class': 'pure-u-1'})
            sub_d = list(map(lambda x: x.text.strip(), sub_d))
        
        sub_data_json[elmnt] = sub_d


#     education = priv_page_soup.body.findAll('div', {'id': 'education'})
#     education = education[0].findAll('div', {'class': 'pure-u-1'})
#     education = list(map(lambda x: x.text.strip(), education))

#     membership = priv_page_soup.body.findAll('div', {'id': 'memberships'})
#     membership = membership[0].findAll('div', {'class': 'pure-u-1'})
#     membership = list(map(lambda x: x.text.strip(), membership))

#     experience = priv_page_soup.body.findAll('div', {'id': 'experience'})
#     experience = experience[0].findAll('div', {'class': 'pure-u-1'})
#     experience = list(map(lambda x: x.text.strip(), experience))

#     registration = priv_page_soup.body.findAll('div', {'id': 'registrations'})
#     registration = registration[0].findAll('div', {'class': 'pure-u-1'})
#     registration = list(map(lambda x: x.text.strip(), registration))
    
#     trainings = priv_page_soup.body.findAll('div', {'id': 'trainings and certifications'})
#     trainings = trainings[0].findAll('div', {'class': 'pure-u-1'})
#     trainings = list(map(lambda x: x.text.strip(), trainings))
    
    
    sub_data_json['claim_check'] = claim_check
    sub_data_json['qualification'] = qualification
    sub_data_json['verification_detail'] = verification_detail
    sub_data_json['profile_score'] = profile_score
    sub_data_json['profile_votes'] = profile_votes
    sub_data_json['exp_years'] = exp_years
#     sub_data_json['education'] = education
#     sub_data_json['membership'] = membership
#     sub_data_json['experience'] = experience
#     sub_data_json['registration'] = registration
#     sub_data_json['trainings'] = trainings
    
    return sub_data_json
    

In [3]:

def scroller(url, name_city, name_prof, no_scrolls, wait_time):
    browser = webdriver.Chrome(executable_path=r"C:\Program Files (x86)\chromedriver.exe")
    browser.get(url.format(n_city = name_city, n_profession = name_prof))
    scroll_try = 5

    # Scroll through the page
    elem = browser.find_element_by_tag_name("body")
    
    last_height = browser.execute_script("return document.body.scrollHeight")
    
    if no_scrolls > 0:
        while no_scrolls:
            time.sleep(wait_time)
            elem.send_keys(Keys.PAGE_DOWN)
            elem.send_keys(Keys.PAGE_DOWN)
            no_scrolls = no_scrolls - 1

    else:
        while scroll_try > 0:
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight - 2000);")
            time.sleep(wait_time)
            new_height = browser.execute_script("return document.body.scrollHeight")
            
            if new_height == last_height:
                scroll_try = scroll_try - 1
                if scroll_try == 1:
                    button = browser.find_elements_by_xpath("//span[@class='u-color--sky-blue u-bold u-c-pointer']")
                    if button != []:
                        button[0].click()
                
            else:
                last_height = new_height
                scroll_try = 5
    
    return browser



In [4]:

# Parse the webpage in BeautifulSoup.
def main_catcher(browser):
    n_id = 1
    page_soup = bsoup(browser.page_source, "html.parser")
    container = page_soup.body.findAll("div", {"class": "u-border-general--bottom"})
    
    print(str(len(container)) + ' records picked from the page, and will be scrapped.')
    
    for contain in container:
        s_contain = contain.findAll("script")
    
        # Convert data in JSON and save
        data_json = json.loads(s_contain[0].string)
        with open(loc + data_city + '_' + str(n_id) + '_' + data_prof + '_' + data_json['name'] + '.json', 'w') as write_file:
            json.dump(data_json, write_file)

        # Parse individual profiles
        sub_data_json = sub_catcher(data_json['url'])
        with open(loc + data_city + '_' + str(n_id) + '_' + data_prof + '_' + sub_data_json['name'] + '_detail.json', 'w') as write_file:
            json.dump(sub_data_json, write_file)
        
        n_id = n_id + 1
    


In [5]:
# main function
start_time = time.time()
print('Started at ' + time.strftime("%H:%M:%S"))

# Launch the webpage
url = 'https://www.practo.com/{n_city}/{n_profession}'
data_city = 'gwalior'
data_prof = 'doctors'
no_scrolls = 0
wait_time = 1
loc = 'C:/Users/Anand/Documents/Data today'

browser = scroller(url, data_city, data_prof, no_scrolls, wait_time)

print(time.strftime("%H:%M:%S") + ': Page has been loaded.')

main_catcher(browser)

browser.close()

end_time = time.time()

print(time.strftime("%H:%M:%S") + ': Data extraction is complete. total time taken: ' + str(round(end_time - start_time, 0)))



Started at 09:09:28
09:10:13: Page has been loaded.
148 records picked from the page, and will be scrapped.
09:13:26: Data extraction is complete. total time taken: 238.0


In [77]:

sub_data_json



{'@context': 'https://schema.org/',
 '@type': 'Physician',
 'name': 'Dr. Madhubala Verma   (PhD)',
 'url': 'https://www.practo.com/gwalior/therapist/madhubala-verma-psychologist',
 'description': 'Psychological Councelling,\nPsychological testing,\nPsychological training',
 'medicalSpecialty': ['Psychologist',
  'Counselling Psychologist',
  'Psychotherapist'],
 'award': [],
 'memberOf': [{'photo': [],
   'openingHours': ['Mo-Sa 17:30-20:00', 'Su 10:00-13:00'],
   '@type': 'MedicalClinic',
   'name': 'Manas Psychological Counselling Center',
   'url': 'https://www.practo.com/gwalior/clinic/manas-psychological-councelling-center-chandak-hospital-14-hospital-road-gwalior-lalitpur-colony',
   'priceRange': 'INR 300',
   'currenciesAccepted': 'INR',
   'address': {'@type': 'PostalAddress',
    'streetAddress': '14- Hospital Road, Landmark: ',
    'addressLocality': 'Lalitpur Colony',
    'addressRegion': 'Gwalior'},
   'image': 'https://www.practo.com/bundles/practopractoapp/images/hospita